In [1]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
import janitor

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

In [3]:
train

,area,majoraxislength,minoraxislength,eccentricity,convexarea,extent,perimeter,target
0,80437,449.454581,232.325506,0.856043,84460,0.674236,1176.305,Kecimen
1,72483,334.417609,282.680889,0.534303,74945,0.706180,1052.159,Kecimen
2,85739,380.370379,288.256159,0.652452,87052,0.762152,1094.576,Kecimen
3,182788,621.206763,379.424446,0.791796,188848,0.733061,1679.075,Besni
4,62835,421.169338,191.169862,0.891051,64406,0.786145,1018.553,Kecimen
...,...,...,...,...,...,...,...,...
715,48945,269.370411,239.162166,0.460121,51456,0.711244,872.289,Kecimen
716,54968,300.954432,234.389570,0.627246,56851,0.751340,893.644,Kecimen
717,166654,607.996465,349.658989,0.818083,169060,0.753518,1574.164,Besni
718,28216,245.401295,150.245582,0.790669,30316,0.622293,683.004,Kecimen


In [4]:
#train = train.drop(["cabin", "name", "passengerid"], axis=1)
#test = test.drop(["cabin", "name", "passengerid"], axis = 1)

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   area             720 non-null    int64  
 1   majoraxislength  720 non-null    float64
 2   minoraxislength  720 non-null    float64
 3   eccentricity     720 non-null    float64
 4   convexarea       720 non-null    int64  
 5   extent           720 non-null    float64
 6   perimeter        720 non-null    float64
 7   target           720 non-null    object 
dtypes: float64(5), int64(2), object(1)
memory usage: 45.1+ KB


In [6]:
cat_col = []
num_col = []
for col in list(train.columns):
    if is_categorical_dtype(train[col]):
        cat_col.append(col)
    elif is_numeric_dtype(train[col]):
        num_col.append(col)

In [7]:
num_col, len(num_col)

(['area',
  'majoraxislength',
  'minoraxislength',
  'eccentricity',
  'convexarea',
  'extent',
  'perimeter'],
 7)

In [8]:
train

,area,majoraxislength,minoraxislength,eccentricity,convexarea,extent,perimeter,target
0,80437,449.454581,232.325506,0.856043,84460,0.674236,1176.305,Kecimen
1,72483,334.417609,282.680889,0.534303,74945,0.706180,1052.159,Kecimen
2,85739,380.370379,288.256159,0.652452,87052,0.762152,1094.576,Kecimen
3,182788,621.206763,379.424446,0.791796,188848,0.733061,1679.075,Besni
4,62835,421.169338,191.169862,0.891051,64406,0.786145,1018.553,Kecimen
...,...,...,...,...,...,...,...,...
715,48945,269.370411,239.162166,0.460121,51456,0.711244,872.289,Kecimen
716,54968,300.954432,234.389570,0.627246,56851,0.751340,893.644,Kecimen
717,166654,607.996465,349.658989,0.818083,169060,0.753518,1574.164,Besni
718,28216,245.401295,150.245582,0.790669,30316,0.622293,683.004,Kecimen


In [9]:
train["target"] = train["target"].replace(["Kecimen", "Besni"], [0,1])

In [10]:
#ordi = OrdinalEncoder(categories=[["L", "M", ])
minmax = MinMaxScaler()
onehot = OneHotEncoder()

In [11]:
#all_col = num_col.copy()
#all_col.append("type")

In [12]:
ct = ColumnTransformer([#("ord", ordi, ["type"]),
                        ("minmax", minmax, num_col)], remainder="passthrough")

In [13]:
#new_ct = ColumnTransformer([("minmax", minmax, ["type")], remainder="passthrough")

In [14]:
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('minmax', MinMaxScaler(),
                                 ['area', 'majoraxislength', 'minoraxislength',
                                  'eccentricity', 'convexarea', 'extent',
                                  'perimeter'])])

In [15]:
train_use = ct.transform(train)
new_features = list(ct.get_feature_names_out())
for i in range(len(new_features)):
    new_features[i] = new_features[i].split("__")[1]
train_df = pd.DataFrame(train_use, columns= new_features)
#train_df["type"] = minmax.fit_transform(np.array(train_df["type"]).reshape(-1,1))
train_df = train_df.clean_names()

In [16]:
test["target"] = test["target"].replace(["Kecimnen", "Besni"], [0,1])
test_use = ct.transform(test)
#new_features = list(ct.get_feature_names_out())
test_df = pd.DataFrame(test_use, columns=new_features)
#test_df["type"] = minmax.fit_transform(np.array(test_df["type"]).reshape(-1,1))

In [17]:
test_df = test_df.clean_names()

In [18]:
train_df.columns

Index(['area', 'majoraxislength', 'minoraxislength', 'eccentricity',
       'convexarea', 'extent', 'perimeter', 'target'],
      dtype='object')

In [19]:
train_df

,area,majoraxislength,minoraxislength,eccentricity,convexarea,extent,perimeter,target
0,0.262568,0.290056,0.254227,0.827058,0.273867,0.646138,0.321550,0.0
1,0.224630,0.140979,0.398693,0.302535,0.229186,0.716253,0.249911,0.0
2,0.287857,0.200529,0.414687,0.495150,0.286038,0.839106,0.274388,0.0
3,0.750744,0.512630,0.676241,0.722318,0.764057,0.775255,0.611673,1.0
4,0.178613,0.253401,0.136156,0.884131,0.179696,0.891770,0.230519,0.0
...,...,...,...,...,...,...,...,...
715,0.112363,0.056684,0.273841,0.181598,0.118885,0.727369,0.146117,0.0
716,0.141090,0.097614,0.260149,0.454058,0.144219,0.815375,0.158440,0.0
717,0.673791,0.495511,0.590847,0.765173,0.671136,0.820156,0.551134,1.0
718,0.013493,0.025622,0.018747,0.720480,0.019615,0.532128,0.036891,0.0


In [20]:
train_df.to_csv("../data/train_fe.csv", index=False)
test_df.to_csv("../data/test_fe.csv", index=False)